# Preprocessing datasets for RGI7 (RGI6 too for testing)

In [1]:
import os 
import glob # finds all the pathnames matching a specified pattern according to the rules used by the Unix shell
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.enums import Resampling

# Import libraries
import rioxarray as riox
from rioxarray.merge import merge_arrays


## Preprocess Millan et al. 2022 dataset

In [ ]:
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Millan_2022/THICKNESS*.tif")

In [ ]:
glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718")
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (50, 50), crs="EPSG:32718", method='max')
merged_raster = merged_raster.round(0).astype("int16")

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Thickness_2022.tif")

## Preprocess Farinotti et al. 2019 dataset

In [ ]:
RGI6 = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6.shp")
RGI6 = RGI6[RGI6.CenLat < -40.5]  # only glaciers in southern Chile-Argentina

# split list
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

# File and folder paths
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Farinotti_2019/RGI*.tif")
list_files_df = pd.DataFrame(list_files,  columns =['dir'])
list_files_df["RGIId"] = list_files_df.dir.str[64:78]

list_files_df = list_files_df[list_files_df['RGIId'].isin(RGI6.RGIId)]
list_files = list_files_df.dir.tolist() # 12000 glaciers

list_files_split = list(split(list_files, 5)) # maximum number of glaciers: 4000 app

In [ ]:
for i in range(0,5): # divide in several grops (5 in this case)
    sublist = list_files_split[i]
    glacier_list = []

    for glacier in sublist:
        glacier_i = riox.open_rasterio(glacier)
        glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
        glacier_list.append(glacier_i)
        
    merged_raster = merge_arrays(dataarrays = glacier_list, res = (50, 50), crs="EPSG:32718", method='max')
    merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/group_" + str(i) + ".tif")
    print(i)

In [ ]:
merged_raster[0].plot()

In [ ]:
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/group_*.tif") # merge 5 groups

glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (100, 100), crs="EPSG:32718", method='max')

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/Thickness_2019.tif")

In [ ]:
for file in list_files: # remove temporary files
    os.remove(file)

## Preprocess Huggonet et al. 2021 dataset

In [ ]:
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt/*.tif")

In [ ]:
glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (100, 100), crs="EPSG:32718", method='max')

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt_2021.tif")

In [2]:
list_files = glob.glob("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt_error/*.tif")

In [4]:
glacier_list = []

# Read rasters file
for glacier in list_files:
    glacier_i = riox.open_rasterio(glacier)
    glacier_i = glacier_i.rio.reproject("EPSG:32718", resampling = Resampling.bilinear)
    glacier_list.append(glacier_i)

# Merge/Mosaic multiple rasters using merge_arrays method of rioxarray
merged_raster = merge_arrays(dataarrays = glacier_list, res = (100, 100), crs="EPSG:32718", method='max')

# Save Raster to disk
merged_raster.rio.to_raster("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/dhdt_error_2021.tif")